<a href="https://colab.research.google.com/github/sunc-dev/Tensorflow-ml-learning/blob/master/Prison_Population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pandas.plotting import scatter_matrix


#google collab libraries
from google.colab import drive

#sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from scipy import stats

In [30]:
#mount google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def load_data(path, options):
  df = pd.read_csv(path, error_bad_lines=options)
  return df

In [33]:
# Define root paths

root_path = '/content/drive/My Drive/Datasets/Crime/'
path = os.path.join(root_path, 'crime_and_incarceration_by_state.csv')
print(path)

crime = load_data(path, False)


/content/drive/My Drive/Datasets/Crime/crime_and_incarceration_by_state.csv
      jurisdiction  includes_jails  year  ...  burglary   larceny vehicle_theft
0          FEDERAL           False  2001  ...       NaN       NaN           NaN
1          ALABAMA           False  2001  ...   40642.0  119992.0       12619.0
2           ALASKA            True  2001  ...    3847.0   16695.0        2618.0
3          ARIZONA           False  2001  ...   54821.0  186850.0       52203.0
4         ARKANSAS           False  2001  ...   22196.0   69590.0        7320.0
..             ...             ...   ...  ...       ...       ...           ...
811       VIRGINIA           False  2016  ...   20159.0  127285.0        9848.0
812     WASHINGTON           False  2016  ...   49249.0  173423.0       32322.0
813  WEST VIRGINIA           False  2016  ...    9127.0   25657.0        2498.0
814      WISCONSIN           False  2016  ...   19498.0   82455.0        9958.0
815        WYOMING           False  2016  ..

In [36]:
# Explore dataset

# Look at top 5 elements
crime.head(5)

# Look at column and data types
crime.info()
crime.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 17 columns):
jurisdiction              816 non-null object
includes_jails            816 non-null bool
year                      816 non-null int64
prisoner_count            816 non-null int64
crime_reporting_change    799 non-null object
crimes_estimated          799 non-null object
state_population          799 non-null float64
violent_crime_total       799 non-null float64
murder_manslaughter       799 non-null float64
rape_legacy               749 non-null float64
rape_revised              199 non-null float64
robbery                   799 non-null float64
agg_assault               799 non-null float64
property_crime_total      799 non-null float64
burglary                  799 non-null float64
larceny                   799 non-null float64
vehicle_theft             799 non-null float64
dtypes: bool(1), float64(11), int64(2), object(3)
memory usage: 102.9+ KB


,year,prisoner_count,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
count,816.000000,816.000000,7.990000e+02,799.000000,799.000000,749.000000,199.000000,799.000000,799.000000,7.990000e+02,799.000000,799.000000,799.000000
mean,2008.500000,28606.033088,6.072322e+06,26228.459324,313.702128,1788.339119,2406.195980,7696.824781,16256.270338,1.878006e+05,40870.408010,127912.287860,19017.904881
std,4.612599,39556.940699,6.725500e+06,33866.838388,386.019821,1865.443299,2550.486639,11107.478615,20849.515589,2.138503e+05,47829.948836,139434.591858,30780.350362
min,2001.000000,1088.000000,4.937540e+05,496.000000,5.000000,99.000000,110.000000,43.000000,270.000000,8.806000e+03,1689.000000,6660.000000,178.000000
25%,2004.750000,5698.000000,1.790026e+06,5213.000000,48.500000,571.000000,780.000000,1106.000000,3529.000000,4.749750e+04,9406.000000,32765.500000,4191.000000
50%,2008.500000,16915.000000,4.314113e+06,15744.000000,179.000000,1238.000000,1723.000000,3933.000000,10083.000000,1.327730e+05,27698.000000,95079.000000,10583.000000
75%,2012.250000,30920.500000,6.808844e+06,31843.000000,429.000000,2092.000000,2680.000000,8702.000000,20308.000000,2.259575e+05,47941.000000,155688.000000,20872.500000
max,2016.000000,216915.000000,3.929648e+07,212867.000000,2503.000000,10198.000000,13702.000000,71142.000000,136087.000000,1.227194e+06,250521.000000,731486.000000,257543.000000


In [48]:
#Preparing the data for visualisations
ids = ['jurisdiction',
       'year'
       ]

dims = ['prisoner_count',
              'state_population',        
              'violent_crime_total',       
              'murder_manslaughter',       
              'rape_legacy',
              'rape_revised',
              'robbery',
              'agg_assault',
              'property_crime_total',
              'burglary',
              'larceny',
              'vehicle_theft'
              ]

print(aggregates)

crime_melt = crime.melt(id_vars = ids,
                        value_vars=dims, 
                        var_name="type",
                        value_name="count")

print(crime_melt)

['prisoner_count', 'state_population', 'violent_crime_total', 'murder_manslaughter', 'rape_legacy', 'rape_revised', 'robbery', 'agg_assault', 'property_crime_total', 'burglary', 'larceny', 'vehicle_theft']
       jurisdiction  year            type     count
0           FEDERAL  2001  prisoner_count  149852.0
1           ALABAMA  2001  prisoner_count   24741.0
2            ALASKA  2001  prisoner_count    4570.0
3           ARIZONA  2001  prisoner_count   27710.0
4          ARKANSAS  2001  prisoner_count   11489.0
...             ...   ...             ...       ...
9787       VIRGINIA  2016   vehicle_theft    9848.0
9788     WASHINGTON  2016   vehicle_theft   32322.0
9789  WEST VIRGINIA  2016   vehicle_theft    2498.0
9790      WISCONSIN  2016   vehicle_theft    9958.0
9791        WYOMING  2016   vehicle_theft     800.0

[9792 rows x 4 columns]


In [0]:
#Prisoner counts per year